In [1]:
import sys
import os
os.chdir("/home/jonfrey/ASL")
sys.path.append("""/home/jonfrey/ASL/src/""")
sys.path.append("""/home/jonfrey/ASL/src/pseudo_label""")

import numpy as np
from visu import Visualizer
import imageio


# STD
import os
import copy

# MISC
import numpy as np
import torch
import imageio
import cv2
from PIL import Image

# matplotlib
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.backends.backend_agg import FigureCanvasAgg
import matplotlib.patches as patches

from PIL import Image
import cv2
import torch
from torchvision import transforms as tf
from utils_asl import LabelLoaderAuto
import numpy as np
from pathlib import Path
import copy
import numpy as np
import pickle


lla = LabelLoaderAuto(root_scannet="/home/jonfrey/Datasets/scannet")

def get_labels(p,sub=1,la="png"):
    p = [str(s) for s in Path(p).rglob(f'*.{la}') if str(s).find("_.png") == -1]
    
    p.sort(
        key=lambda x: int(x.split("/")[-1][:-4])
    )
    
    return p[::sub]

def get_labels_multiple( base_name, scenes_list,sub=1, test=False, train=False):
    labels = []
    for s in scenes_list:
        labels += get_labels(base_name.replace("xxxx",s) ,sub=sub)
    
    if test: 
        return labels[int( len(labels)*0.8):]
    
    if train:
        return labels[:int( len(labels)*0.8)]
    
    return labels



class AccMonitor():
    def __init__(self):
        self.metrics = {}
        self.metric_dict = {'correct': 0,
                            'correct_valid': 0,
                            'correct_valid_both': 0,
                            'total': 0, 
                            'total_valid':0,
                            'total_valid_both':0,
                            'images': 0}
    def register(self,name):
        self.metrics[name] = copy.deepcopy(self.metric_dict)
        
    def update(self,name, label, gt):
        if name not in self.metrics.keys():
            self.register(name)
        m1 = gt != -1
        self.metrics[name]["total_valid"] += (m1).sum()
        self.metrics[name]["correct_valid"] += np.sum(gt[m1] == label[m1])
        
        m2 = (gt != -1 )*(label != -1)
        self.metrics[name]["total_valid_both"] += (m2).sum()
        self.metrics[name]["correct_valid_both"] += np.sum(gt[m2] == label[m2])
        
        self.metrics[name]["total"] += gt.size
        self.metrics[name]["correct"] += np.sum(gt == label)
        self.metrics[name]["images"] += 1
    
    def store_pickle(self, p):
        with open(p, 'wb') as handle:
            pickle.dump(self.metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def restore_pickle(self, p):
        with open(p, 'rb') as handle:
            self.metrics = pickle.load(handle)
            
    def __str__(self):
        s = "Acc-Monitor:\n"
        for k in self.metrics.keys():
            acc = self.metrics[k]['correct_valid'] / self.metrics[k]['total_valid']
            acc2 = self.metrics[k]['correct_valid_both'] / self.metrics[k]['total_valid_both']
            s += f"  {k}: Nr-Images: {self.metrics[k]['images']} , Avg-Acc: {acc}  Avg-Acc: valid both {acc2} \n"  
        return s

In [3]:
gt_p = "/home/jonfrey/Datasets/scannet/scans/xxxx/label-filt"

data = []
data.append( [ 1, 3, f"/home/jonfrey/Datasets/labels_generated/labels_pretrained_2/scans/xxxx", "iter1_pred"])
data.append( [ 1, 3, f"/home/jonfrey/Datasets/labels_generated/labels_4_scenes_map_1/scans/xxxx" ,"iter1_repro" ])
data.append( [ 2, 3, f"/home/jonfrey/Datasets/labels_generated/iteration2_individual_no_replay/scans/xxxx", "iter2_0_pred"])
data.append( [ 2, 3, f"/home/jonfrey/Datasets/labels_generated/iteration2_individual_no_replay_reprojected/scans/xxxx", "iter2_0_repro"])
data.append( [ 2, 3, f"/home/jonfrey/Datasets/labels_generated/iteration_2_individual_replay02/scans/xxxx", "iter2_02_pred"])
data.append( [ 2, 3, f"/home/jonfrey/Datasets/labels_generated/iteration_2_individual_replay02_reprojected/scans/xxxx", "iter2_02_repro"])
data.append( [ 2, 3, f"/home/jonfrey/Datasets/labels_generated/iteration_2_individual_replay05/scans/xxxx", "iter2_05_pred"])
data.append( [ 2, 3, f"/home/jonfrey/Datasets/labels_generated/iteration_2_individual_replay09/scans/xxxx", "iter2_9_pred"])

data.append( [ 2, 3, f"/home/jonfrey/Datasets/labels_generated/labels_iteration_3_individual_replay02_reprojected/scans/xxxx" , "iter3_02_repro"])
data.append( [ 3, 3, f"/home/jonfrey/Datasets/labels_generated/labels_iteration_3_individual_replay02/scans/xxxx" , "iter3_02_pred"])
data.append( [ 1, 5, f"/home/jonfrey/Datasets/labels_generated/labels_iter1_5cm_reprojected/scans/xxxx" , "iter1_repro_5cm"])
data.append( [ 1, 5, f"/home/jonfrey/Datasets/labels_generated/labels_gt_reprojected/scans/xxxx" , "gt_repro_5cm"])
data.append( [ 1, 3, f"/home/jonfrey/Datasets/labels_generated/labels_gt_reprojected_3cm/scans/xxxx" , "gt_repro_3cm"])
data.append( [ 1, 5, f"/home/jonfrey/Datasets/labels_generated/labels_iter2_5cm_replay00/scans/xxxx" , "iter2_00_5cm_pred"])
data.append( [ 1, 5, f"/home/jonfrey/Datasets/labels_generated/labels_iter2_5cm_replay02/scans/xxxx" , "iter2_02_5cm_pred"])
data.append( [ 1, 5, f"/home/jonfrey/Datasets/labels_generated/labels_iter2_5cm_rep02_reprojected/scans/xxxx" , "iter2_02_5cm_repro"])

data.append( [ 1, 5, f"/home/jonfrey/Datasets/labels_generated/labels_iter2_5cm_replay02_e20/scans/xxxx" , "iter2_02_20_epochs_5cm_pred"])
scenes = [["scene0000_00",
        "scene0000_01",
        "scene0000_02"],
        ["scene0001_00",
        "scene0001_01"],
        ["scene0002_00",
        "scene0002_01"],
        ["scene0003_00",
        "scene0003_01",
       "scene0003_02"],
        ["scene0004_00"],
        ["scene0005_00",
        "scene0005_01"]]

data_arr = np.array( data )


In [4]:
def array_to_acc( data_arr, gt_p, scenes , mode = "test", force=True, clean=False):
    NR = data_arr.shape[0]
    
    if clean: 
        for i in range(NR):
            path = data_arr[i,3]+"_"+ mode +"_res.pkl"
            try:
                os.remove(path)
            except:
                pass
    if mode == "test":
        test = True
    else:
        test = False
    
    if mode == "train":
        train = True
    else:
        train = False
    
    acc = np.zeros( (NR, len(scenes)) )
    
    for i in range(NR):
        path = data_arr[i,3]+"_"+ mode +"_res.pkl"
        print(path)
        if os.path.isfile(path) and not force:
                with open(path, 'rb') as handle:
                    acc[i] = pickle.load(handle)
                
        else:
        
            for nr_s, s in enumerate( scenes):
                accm = AccMonitor() 
                print(data_arr[i,2])
                print(s)
                gt_paths = get_labels_multiple( gt_p, s ,10, test, train)
                ppp = get_labels_multiple( data_arr[i,2] , s ,1, test, train)
                print( len( ppp ) )
                if len( ppp) != len(gt_paths):
                    print("ERROR break")
                    return False
                
                
                for j, gtp in enumerate( gt_paths):
                    gt = lla.get( gtp )[0]
                    tmp = lla.get( ppp[j] )[0]
                    accm.update("acc", tmp-1 ,gt-1)
                    # if j == 10:
                    #    print(j)
                    #    break
                acc[i,nr_s] = accm.metrics["acc"]['correct_valid'] / accm.metrics["acc"]['total_valid']

                
            with open(path, 'wb') as handle:
                pickle.dump(acc[i], handle, protocol=pickle.HIGHEST_PROTOCOL)
                print(acc)

    return acc

results_acc = array_to_acc( data_arr, gt_p, scenes , "test", False, False)

iter1_pred_test_res.pkl
iter1_repro_test_res.pkl
iter2_0_pred_test_res.pkl
iter2_0_repro_test_res.pkl
iter2_02_pred_test_res.pkl
iter2_02_repro_test_res.pkl
iter2_05_pred_test_res.pkl
iter2_9_pred_test_res.pkl
iter3_02_repro_test_res.pkl
iter3_02_pred_test_res.pkl
iter1_repro_5cm_test_res.pkl
gt_repro_5cm_test_res.pkl
gt_repro_3cm_test_res.pkl
iter2_00_5cm_pred_test_res.pkl
iter2_02_5cm_pred_test_res.pkl
iter2_02_5cm_repro_test_res.pkl
iter2_02_20_epochs_5cm_pred_test_res.pkl
/home/jonfrey/Datasets/labels_generated/labels_iter2_5cm_replay02_e20/scans/xxxx
['scene0000_00', 'scene0000_01', 'scene0000_02']
354
/home/jonfrey/Datasets/labels_generated/labels_iter2_5cm_replay02_e20/scans/xxxx
['scene0001_00', 'scene0001_01']
57
/home/jonfrey/Datasets/labels_generated/labels_iter2_5cm_replay02_e20/scans/xxxx
['scene0002_00', 'scene0002_01']
250
/home/jonfrey/Datasets/labels_generated/labels_iter2_5cm_replay02_e20/scans/xxxx
['scene0003_00', 'scene0003_01', 'scene0003_02']
96
/home/jonfrey/Dat

In [6]:
import pandas as pd
names = data_arr[:,-1].tolist()
df = pd.DataFrame(data= results_acc[:,:-1].T, columns=names)
df.mean()

d2 = np.zeros( (results_acc[:,:-1].shape[0],results_acc[:,:-1].shape[1]+1 ) )
d2[:,:-1] = results_acc[:,:-1]
d2[:,-1] = df.mean()

df2 = pd.DataFrame(data= d2.T, columns=names )

print("iter_1 == pretrained network output")
print("iter_1_reprojected == pseudo label reprojected")
print("cl -> network output;  XX percentage of replayed samples \n \n")

print( df.mean() )
#print( df.std() )

df2

iter_1 == pretrained network output
iter_1_reprojected == pseudo label reprojected
cl -> network output;  XX percentage of replayed samples 
 

iter1_pred                     0.509018
iter1_repro                    0.574950
iter2_0_pred                   0.564507
iter2_0_repro                  0.567591
iter2_02_pred                  0.569138
iter2_02_repro                 0.571338
iter2_05_pred                  0.559408
iter2_9_pred                   0.530253
iter3_02_repro                 0.542384
iter3_02_pred                  0.552132
iter1_repro_5cm                0.577131
gt_repro_5cm                   0.925080
gt_repro_3cm                   0.934849
iter2_00_5cm_pred              0.548650
iter2_02_5cm_pred              0.553335
iter2_02_5cm_repro             0.563247
iter2_02_20_epochs_5cm_pred    0.541114
dtype: float64


,iter1_pred,iter1_repro,iter2_0_pred,iter2_0_repro,iter2_02_pred,iter2_02_repro,iter2_05_pred,iter2_9_pred,iter3_02_repro,iter3_02_pred,iter1_repro_5cm,gt_repro_5cm,gt_repro_3cm,iter2_00_5cm_pred,iter2_02_5cm_pred,iter2_02_5cm_repro,iter2_02_20_epochs_5cm_pred
0,0.604986,0.702035,0.717161,0.744377,0.716350,0.735990,0.720569,0.682502,0.732365,0.720674,0.715534,0.942653,0.952927,0.725416,0.720906,0.736860,0.717174
1,0.485094,0.537268,0.515862,0.470219,0.529288,0.494420,0.498215,0.417316,0.406984,0.474861,0.534082,0.948616,0.950438,0.428223,0.455272,0.470114,0.407660
2,0.305407,0.379698,0.366709,0.386879,0.367548,0.393875,0.367189,0.349569,0.379108,0.367435,0.374544,0.887161,0.926547,0.363498,0.362313,0.381418,0.358097
3,0.746380,0.815898,0.813035,0.815194,0.805783,0.809488,0.789175,0.792326,0.802367,0.796995,0.813634,0.959801,0.976596,0.807545,0.801219,0.804275,0.799358
4,0.403223,0.439852,0.409769,0.421286,0.426717,0.422919,0.421893,0.409551,0.391094,0.400696,0.447860,0.887168,0.867738,0.418569,0.426968,0.423566,0.423280
5,0.509018,0.574950,0.564507,0.567591,0.569138,0.571338,0.559408,0.530253,0.542384,0.552132,0.577131,0.925080,0.934849,0.548650,0.553335,0.563247,0.541114


In [6]:
(df2*100).round(1)

,iter1_pred,iter1_repro,iter2_0_pred,iter2_0_repro,iter2_02_pred,iter2_02_repro,iter2_05_pred,iter2_9_pred,iter3_02_repro,iter3_02_pred,iter1_repro_5cm,gt_repro_5cm,gt_repro_3cm,iter2_00_5cm_pred,iter2_02_5cm_pred,iter2_02_5cm_repro
0,60.5,70.2,71.7,74.4,71.6,73.6,72.1,68.3,73.2,72.1,71.6,94.3,95.3,72.5,72.1,73.7
1,48.5,53.7,51.6,47.0,52.9,49.4,49.8,41.7,40.7,47.5,53.4,94.9,95.0,42.8,45.5,47.0
2,30.5,38.0,36.7,38.7,36.8,39.4,36.7,35.0,37.9,36.7,37.5,88.7,92.7,36.3,36.2,38.1
3,74.6,81.6,81.3,81.5,80.6,80.9,78.9,79.2,80.2,79.7,81.4,96.0,97.7,80.8,80.1,80.4
4,40.3,44.0,41.0,42.1,42.7,42.3,42.2,41.0,39.1,40.1,44.8,88.7,86.8,41.9,42.7,42.4
5,50.9,57.5,56.5,56.8,56.9,57.1,55.9,53.0,54.2,55.2,57.7,92.5,93.5,54.9,55.3,56.3


In [ ]:
visu = Visualizer( p_visu= "/home/jonfrey/Pictures/test")    
    
gt_paths = get_labels_multiple( gt_p, ["scene0000_00"] ,10, False,False)
ppp = get_labels_multiple( data_arr[i,2] , ["scene0000_00"] ,1, False,False)

In [ ]:
target = "labels_iter2_5cm_replay02"
base = "/home/jonfrey/Datasets/labels_generated/" + target + "/scans"

for s in [x for x in os.listdir(base)]:
    print(s)
    b =  os.path.join(base,s)
    bb = os.listdir( b)[0]
    src = os.path.join(b,bb)
    dst = os.path.join(b,target)
    print( src, dst)
    os.rename(src, dst)

In [ ]:
import shutil 
target = "iteration_2_individual_replay09"
base = "/home/jonfrey/Datasets/labels_generated/" + target
for s in [x[0] for x in os.walk(base) if x[0].split("/")[-1][:7] ==  "scene00"] :
    dst = os.path.join(base, "scans")
    
    try:
        pass
        #shutil.move(s, dst)
    except:
        pass